In [1]:
from ptgnn.runtime_config.run_config import fetch_loaders

In [2]:
data_config = {
    'dataset' : {
        'type' : "rs",
        'mask_chiral_tags': True,
        'transformation_mode': 'chienn_tree_basic',
    },
    'loader': {
        'general': {
            'n_neighbors_in_circle': 3,
            'batch_size': 32,
            'num_workers': 0,
        },
        'train': {},
        'test': {},
        'val': {}
    }
}

In [3]:
_, _, val_loader = fetch_loaders(data_config=data_config)

In [4]:
for batch in val_loader:
    display(batch)
    break

DataBatch(x=[2356, 118], edge_index=[2, 6416], edge_attr=[6416, 80], pos=[2356, 6], y=[32], batch=[2356], ptr=[33], ptree=[2356])

# model for one type (Z)

In [5]:
# put through linear layers
# todo: do so only for a  few and not for all as not all needed for this module

In [6]:
k = 3
hidden_dim = 128

In [7]:
import torch

In [8]:
k_list = torch.nn.ModuleList([
    torch.nn.Linear(118, hidden_dim)
    for _ in range(k)
])

In [9]:
# put all elements through linear layers
# todo add masking to only do to relevant layers
elem_z_put_though = [
    k_elem(batch.x)
    for k_elem in k_list
]

In [10]:
elem_z_put_though

[tensor([[ 0.4958, -0.0202,  0.0125,  ..., -0.0793,  0.2432, -0.3656],
         [ 0.0668,  0.0434, -0.1639,  ...,  0.1165, -0.0997, -0.1146],
         [ 0.1170,  0.1148,  0.0455,  ...,  0.1200, -0.1237, -0.1449],
         ...,
         [ 0.3406, -0.0279,  0.0297,  ..., -0.1391,  0.0480, -0.0959],
         [-0.1321,  0.2321, -0.1100,  ...,  0.0879, -0.2900, -0.1240],
         [ 0.3406, -0.0279,  0.0297,  ..., -0.1391,  0.0480, -0.0959]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.0681, -0.0634, -0.0257,  ..., -0.0493, -0.3378,  0.0891],
         [-0.0965, -0.0736, -0.0091,  ..., -0.2011, -0.1737, -0.0101],
         [-0.1231, -0.1310, -0.0694,  ..., -0.0488, -0.2585,  0.0721],
         ...,
         [-0.1469, -0.1089,  0.0062,  ...,  0.0093, -0.2137, -0.0456],
         [-0.1384, -0.0245,  0.0967,  ..., -0.0007, -0.2729,  0.0533],
         [-0.1469, -0.1089,  0.0062,  ...,  0.0093, -0.2137, -0.0456]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.3117, -0.0896,  0.0070,  ...,  0.00

In [8]:
import json

In [9]:
# extract permutation trees from string
permutation_trees = [
    json.loads(p_string)
    for p_string in batch.ptree
]

In [10]:
permutation_trees

[{'P': [1]},
 {'P': [0, {'Z': [3, 7, 5]}]},
 {'P': [3, {'Z': [0, 5, 7]}]},
 {'P': [2, {'Z': [9]}]},
 {'P': [5, {'Z': [0, 7, 3]}]},
 {'P': [4, {'Z': [14, 19]}]},
 {'P': [7, {'Z': [0, 3, 5]}]},
 {'P': [6]},
 {'P': [9, {'Z': [2]}]},
 {'P': [8, {'Z': [11, 13]}]},
 {'P': [11, {'Z': [8, 13]}]},
 {'P': [10, {'Z': [15, 17]}]},
 {'P': [13, {'Z': [8, 11]}]},
 {'P': [12]},
 {'P': [15, {'Z': [10, 17]}]},
 {'P': [14, {'Z': [4, 19]}]},
 {'P': [17, {'Z': [10, 15]}]},
 {'P': [16]},
 {'P': [19, {'Z': [4, 14]}]},
 {'P': [18, {'Z': [21, 23, 25]}]},
 {'P': [21, {'Z': [18, 25, 23]}]},
 {'P': [20, {'Z': [27, 31, 29]}]},
 {'P': [23, {'Z': [18, 21, 25]}]},
 {'P': [22, {'Z': [26, 35, 33]}]},
 {'P': [25, {'Z': [18, 23, 21]}]},
 {'P': [24]},
 {'P': [27, {'Z': [20, 29, 31]}]},
 {'P': [26, {'Z': [22, 33, 35]}]},
 {'P': [29, {'Z': [20, 31, 27]}]},
 {'P': [28]},
 {'P': [31, {'Z': [20, 27, 29]}]},
 {'P': [30]},
 {'P': [33, {'Z': [22, 35, 26]}]},
 {'P': [32]},
 {'P': [35, {'Z': [22, 26, 33]}]},
 {'P': [34]},
 {'P': [3

In [14]:
def depth(d):
    if isinstance(d, list):
        return max(map(depth, d))
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [62]:
# get number of layers
num_layers = depth(permutation_trees)
num_layers

2

In [ ]:
def get_at_depth(d: object, depth: int = 1) -> list:
    if depth == 1:
        # return the values
        return list(d.values())
    else:
        return [
            *get_at_depth(child, depth-1)
            for child in d.values()
        ]

In [ ]:
# get actions to take at level max
actions = [

    for tree in permutation_trees
]

In [74]:
def get_matrix(prefix, tree):
    if isinstance(tree, list):
        return torch.tensor()


            list(map(get_matrix, tree))
    #elif isinstance(tree, dict):
    #    key = next(iter(tree.keys()))
    #    return get_matrix(tree[key])
    elif isinstance(tree, int):
        return tree
    else:
        return 0


In [76]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=-1)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp

tensor([   0,    0,    1,  ...,    0, 2354,   -1])

In [88]:
def get_matrix(tree, depth, prefix: list = []) -> list:
    if isinstance(tree, list):
        return torch.cat([
            get_matrix(child, depth, prefix + [idx])
            for idx, child in enumerate(tree)
        ], dim=0)

    elif isinstance(tree, dict):
        key = next(iter(tree.keys()))
        return get_matrix(tree[key], depth-1, prefix)

    elif isinstance(tree, int):
        return torch.tensor(prefix + [tree] + [-1]*depth).reshape(1, -1)

    else:
        return [-1]

temp = get_matrix(permutation_trees, depth=num_layers)
temp

tensor([[   0,    0,    1,   -1],
        [   1,    0,    0,   -1],
        [   1,    1,    0,    3],
        ...,
        [2354,    1,    1, 2351],
        [2354,    1,    2, 2353],
        [2355,    0, 2354,   -1]])

In [91]:
permutation_trees[1]

{'P': [0, {'Z': [3, 7, 5]}]}

In [86]:
torch.cat([torch.tensor([[1,2,3]]), torch.tensor([[1,2,3], [2,3,4],[5,6,7]])], dim=0)

tensor([[1, 2, 3],
        [1, 2, 3],
        [2, 3, 4],
        [5, 6, 7]])

In [77]:
selection = temp[0]
print(selection)
print(type(selection))
print(len(selection))

[1]
<class 'list'>
1


In [65]:
[-1]*0

[]

# simplistic version:

In [11]:
z_layer = torch.nn.ModuleList([
    torch.nn.Linear(118, 118)
    for _ in range(k)
])
z_final_layer = torch.nn.Linear(118,118)
z_elu = torch.nn.ELU()

In [12]:
p_layer = torch.nn.Linear(118, 118)
p_final_layer = torch.nn.Linear(118, 118)

In [13]:
def fetch_or_produce(subtree):
    if isinstance(subtree, int):
        return batch.x[subtree]
    else:
        return fulfill_module(subtree)

In [14]:
def fulfill_module(tree):
    # get node type
    node_type = next(iter(tree.keys()))

    # iterate over indices/trees and replace by data
    data_list = [
        fetch_or_produce(subtree)
        for subtree in tree[node_type]
    ]

    data_list = torch.stack(data_list)

    # put through layer
    if node_type == "P":
        # run through layer for each input
        after_p = p_layer(data_list)
        # sum up
        after_p = torch.sum(after_p, dim=0)
        # put through final linear layer
        after_p = p_final_layer(after_p)

        return after_p

    elif node_type == "Z":
        # run through embedding layers
        after_z = [
            layer(data_list)
            for layer in z_layer
        ]
        # shift
        _idx = torch.arange(data_list.shape[0])
        after_z = [
            embedding[torch.roll(_idx, shifts=idx), :]
            for idx, embedding in enumerate(after_z)
        ]
        after_z = torch.stack(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        # apply ELU
        after_z = z_elu(after_z)
        # layer
        after_z = z_final_layer(after_z)
        # sum
        after_z = torch.sum(after_z, dim=0)
        return after_z


In [15]:
torch.tensor([1,2,3,4])[torch.roll(torch.arange(4), shifts=1)]

tensor([4, 1, 2, 3])

In [17]:
out = torch.stack([
    fulfill_module(ptree)
    for ptree in permutation_trees
])
display(out)
display(out.shape)

tensor([[-0.0117, -0.0078, -0.2394,  ...,  0.0091, -0.0309,  0.0926],
        [ 0.3458,  0.2275, -0.4567,  ..., -0.1546, -0.5310, -0.1610],
        [ 0.2885,  0.1240, -0.4007,  ..., -0.1202, -0.6570, -0.0793],
        ...,
        [-0.0008, -0.0161, -0.1960,  ..., -0.0736, -0.0838,  0.0907],
        [ 0.3346,  0.1963, -0.4824,  ..., -0.0510, -0.5311, -0.2466],
        [-0.0008, -0.0161, -0.1960,  ..., -0.0736, -0.0838,  0.0907]],
       grad_fn=<StackBackward0>)

torch.Size([2356, 118])

In [97]:
new_node_values = []
for ptree in permutation_trees:
    new_node_values.append(
        fulfill_module(ptree)
    )

{'P': [1]}
{'P': [0, {'Z': [3, 7, 5]}]}
{'P': [3, {'Z': [0, 5, 7]}]}
{'P': [2, {'Z': [9]}]}
{'P': [5, {'Z': [0, 7, 3]}]}
{'P': [4, {'Z': [14, 19]}]}
{'P': [7, {'Z': [0, 3, 5]}]}
{'P': [6]}
{'P': [9, {'Z': [2]}]}
{'P': [8, {'Z': [11, 13]}]}
{'P': [11, {'Z': [8, 13]}]}
{'P': [10, {'Z': [15, 17]}]}
{'P': [13, {'Z': [8, 11]}]}
{'P': [12]}
{'P': [15, {'Z': [10, 17]}]}
{'P': [14, {'Z': [4, 19]}]}
{'P': [17, {'Z': [10, 15]}]}
{'P': [16]}
{'P': [19, {'Z': [4, 14]}]}
{'P': [18, {'Z': [21, 23, 25]}]}
{'P': [21, {'Z': [18, 25, 23]}]}
{'P': [20, {'Z': [27, 31, 29]}]}
{'P': [23, {'Z': [18, 21, 25]}]}
{'P': [22, {'Z': [26, 35, 33]}]}
{'P': [25, {'Z': [18, 23, 21]}]}
{'P': [24]}
{'P': [27, {'Z': [20, 29, 31]}]}
{'P': [26, {'Z': [22, 33, 35]}]}
{'P': [29, {'Z': [20, 31, 27]}]}
{'P': [28]}
{'P': [31, {'Z': [20, 27, 29]}]}
{'P': [30]}
{'P': [33, {'Z': [22, 35, 26]}]}
{'P': [32]}
{'P': [35, {'Z': [22, 26, 33]}]}
{'P': [34]}
{'P': [37]}
{'P': [36, {'Z': [39, 41, 43]}]}
{'P': [39, {'Z': [36, 43, 41]}]}
{'P